In [1]:
import snscrape.modules.twitter as sntwitter   
import pandas as pd

In [ ]:
import datetime

time = datetime.datetime.fromtimestamp(int('1502942400000')/1000).strftime('%Y/%m/%d %H:%M:%S')  # 僅轉換格式讓自己看懂而已
time

In [2]:
from binance.client import Client

In [35]:
from binance.client import Client
import configparser

config = configparser.ConfigParser()
config.read('config.ini')  # 讀取 config.ini 避免 api key 和 api secret 暴露在程式中

api_key = config['binance']['api_key']
api_secret = config['binance']['api_secret']

client = Client(api_key,api_secret)


btc_price = client.get_symbol_ticker(symbol='BTCUSDT')   # 讀取最近一筆 BTC 價格
print(btc_price)

{'symbol': 'BTCUSDT', 'price': '16732.40000000'}


In [6]:
timestamp = client._get_earliest_valid_timestamp('BTCUSDT','1d')  # 找到最早一筆交易時間

In [36]:
bars_BTC = client.get_historical_klines('BTCUSDT', '1d', timestamp)  # 將幣價以每天一紀錄
bars_ETH = client.get_historical_klines('ETHUSDT', '1d', timestamp)
bars_SOL = client.get_historical_klines('SOLUSDT', '1d', timestamp)
bars_DOGE = client.get_historical_klines('DOGEUSDT', '1d', timestamp)

In [37]:
# 利用 pandas 整理需要的欄位
btc_df = pd.DataFrame(bars_BTC)
btc_df.drop(columns=[7,8,9,10,11],inplace=True)
btc_df.set_axis(['Open time','Open','High','Low','Close','Volume','Close time'],axis=1,inplace = True)

btc_close = pd.to_numeric(btc_df['Close'])  # 將str轉為 num
btc_close_time = pd.to_datetime(btc_df['Close time'], unit='ms')  # 改變時間表示法

eth_df = pd.DataFrame(bars_ETH)
eth_df.drop(columns=[7,8,9,10,11],inplace=True)
eth_df.set_axis(['Open time','Open','High','Low','Close','Volume','Close time'],axis=1,inplace = True)

eth_close = pd.to_numeric(eth_df['Close'])
eth_close_time = pd.to_datetime(eth_df['Close time'], unit='ms')

sol_df = pd.DataFrame(bars_SOL)
sol_df.drop(columns=[7,8,9,10,11],inplace=True)
sol_df.set_axis(['Open time','Open','High','Low','Close','Volume','Close time'],axis=1,inplace = True)

sol_close = pd.to_numeric(sol_df['Close'])
sol_close_time = pd.to_datetime(sol_df['Close time'], unit='ms')

doge_df = pd.DataFrame(bars_DOGE)
doge_df.drop(columns=[7,8,9,10,11],inplace=True)
doge_df.set_axis(['Open time','Open','High','Low','Close','Volume','Close time'],axis=1,inplace = True)

doge_close = pd.to_numeric(doge_df['Close'])
doge_close_time = pd.to_datetime(doge_df['Close time'], unit='ms')  # 改變時間表示法

/tmp/ipykernel_32589/1962537283.py:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  btc_df.set_axis(['Open time','Open','High','Low','Close','Volume','Close time'],axis=1,inplace = True)
/tmp/ipykernel_32589/1962537283.py:11: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  eth_df.set_axis(['Open time','Open','High','Low','Close','Volume','Close time'],axis=1,inplace = True)
/tmp/ipykernel_32589/1962537283.py:18: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  sol_df.set_axis(['Open time','Open','High','Low','Close','Volume','Close time'],axis=1,inplace = True)
/tmp/ipykernel_32589/1962537283.py:25: FutureWarning: DataFrame.set_axis 'inplace' keyword is depr

In [14]:
btc_close_time, close_price

(0       2017-08-17 04:59:59.999
 1       2017-08-17 05:59:59.999
 2       2017-08-17 06:59:59.999
 3       2017-08-17 07:59:59.999
 4       2017-08-17 08:59:59.999
                   ...          
 47020   2023-01-02 15:59:59.999
 47021   2023-01-02 16:59:59.999
 47022   2023-01-02 17:59:59.999
 47023   2023-01-02 18:59:59.999
 47024   2023-01-02 19:59:59.999
 Name: Close time, Length: 47025, dtype: datetime64[ns],
 0         4308.83
 1         4315.32
 2         4324.35
 3         4349.99
 4         4360.69
            ...   
 47020    16734.66
 47021    16710.88
 47022    16715.42
 47023    16723.81
 47024    16723.32
 Name: Close, Length: 47025, dtype: float64)

In [16]:
type(btc_close_time)

pandas.core.series.Series

In [58]:
def get_time(close_time,close_price):
    time = []
    for i in range(len(close_time)):
        if close_price.iloc[i] <= close_price.iloc[i-1]*0.9:
            time.append([str(btc_close_time.iloc[i-1]).split(' ')[0],str(btc_close_time.iloc[i]).split(' ')[0],str(btc_close_time.iloc[i+1]).split(' ')[0]])
        
        elif close_price.iloc[i] >= close_price.iloc[i-1]*1.10:
            time.append([str(btc_close_time.iloc[i-1]).split(' ')[0],str(btc_close_time.iloc[i]).split(' ')[0],str(btc_close_time.iloc[i+1]).split(' ')[0]])
        

    return time

In [42]:
len(btc_close_time)

1965

In [59]:
btc_time = get_time(btc_close_time, btc_close)
eth_time = get_time(eth_close_time, eth_close)
sol_time = get_time(sol_close_time, sol_close)
doge_time = get_time(doge_close_time, doge_close)


In [61]:
len(btc_time)

70

In [41]:
len(btc_time)

319

In [33]:
import snscrape.modules.twitter as sntwitter
import snscrape

In [ ]:
def get_date(row):
    row = str(row)
    return row.split(' ')[0]

In [90]:
btc_tweets = []
for time in btc_time:
    n=0
    query = f'BTC or Bitcoin since:{time[0]} until:{time[1]}'
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if n == 1000:
            break
        else:
            btc_tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.user.followersCount, tweet.user.friendsCount, tweet.user.statusesCount]) 
            n+=1
btc_df = pd.DataFrame(btc_tweets, columns=['Date','User','Content','Followers','Friends','Statuses'])
btc_df['Date'] = btc_df['Date'].apply(get_date)
btc_df.to_csv('BTC_tweets.csv')

In [91]:
eth_tweets = []
for time in eth_time:
    n=0
    query = f'ETH or Ethereum since:{time[0]} until:{time[1]}'
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if n == 1000:
            break
        else:
            eth_tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.user.followersCount, tweet.user.friendsCount, tweet.user.statusesCount]) 
            n+=1
eth_df = pd.DataFrame(eth_tweets, columns=['Date','User','Content','Followers','Friends','Statuses'])
eth_df['Date'] = eth_df['Date'].apply(get_date)
eth_df.to_csv('ETH_tweets.csv')

In [92]:
sol_tweets = []
for time in sol_time:
    n=0
    query = f'SOL or Solana since:{time[0]} until:{time[1]}'
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if n == 1000:
            break
        else:
            sol_tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.user.followersCount, tweet.user.friendsCount, tweet.user.statusesCount]) 
            n+=1
sol_df = pd.DataFrame(sol_tweets, columns=['Date','User','Content','Followers','Friends','Statuses'])
sol_df['Date'] = sol_df['Date'].apply(get_date)
sol_df.to_csv('SOL_tweets.csv')

In [93]:
doge_tweets = []
for time in doge_time:
    n=0
    query = f'DOGE or Dogecoin since:{time[0]} until:{time[1]}'
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if n == 1000:
            break
        else:
            doge_tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.user.followersCount, tweet.user.friendsCount, tweet.user.statusesCount]) 
            n+=1
doge_df = pd.DataFrame(doge_tweets, columns=['Date','User','Content','Followers','Friends','Statuses'])
doge_df['Date'] = doge_df['Date'].apply(get_date)
doge_df.to_csv('DOGE_tweets.csv')

In [77]:
def get_date(row):
    row = str(row)
    return row.split(' ')[0]

In [79]:
df

,Date,User,Content,Followers,Friends,Statuses
0,2017-09-13 23:46:51+00:00,BMouler,Mike Krieger Asks Which Is 'Fraudulent': Bitco...,6922,586,69073
1,2017-09-13 23:15:01+00:00,big_blockers,The current price of Bitcoin is $3941.87.\n Th...,3041,165,107635
2,2017-09-13 23:02:05+00:00,theperson3233,@officialmcafee You talking about Bitcoin cash...,397,610,4528
3,2017-09-13 23:01:03+00:00,DisruptBanksy,Buy or sell $100 of #digitalcurrency or more v...,4490,4974,148694
4,2017-09-13 22:50:57+00:00,JasperJove,"@officialmcafee Hi John, when you say Bitcoin ...",182,555,7548
...,...,...,...,...,...,...
685,2022-11-09 23:44:37+00:00,streetstryker,@MattStirner @saylor We talking bout the dolla...,76,31,439
686,2022-11-09 23:41:03+00:00,JoeJoe6817835,"Guns don’t kill people, people do. Is your #BT...",49,127,28
687,2022-11-09 23:36:11+00:00,EmanFT_76,If the #BTC or #Bitcoin price were actually ba...,110,199,31
688,2022-11-09 23:34:30+00:00,0xZander20,WWYD? Buy another ASIC or #BTMFD? #BTC #Bitcoin,115,239,32


In [81]:
df['Date'] = df['Date'].apply(get_date)

In [86]:
df[df['Followers']>5000]

,Date,User,Content,Followers,Friends,Statuses
0,2017-09-13,BMouler,Mike Krieger Asks Which Is 'Fraudulent': Bitco...,6922,586,69073
14,2017-09-14,CryptoNewswire,Buy Gold Or Bitcoin? - https://t.co/MdG3c38g8X...,9787,175,278629
53,2017-12-05,NODEfather,"So @RSKsmart is great... But remember folks, I...",5059,967,34192
57,2017-12-05,Freemason_UK,If you’re new to Crypto and fancy buying some ...,7650,3710,47729
58,2017-12-05,NYCPunter,@BankofGSimms Bitcoin knows no territorial bou...,10479,1738,82861
...,...,...,...,...,...,...
616,2022-02-03,Mad_MaxCrypto,"BTC: ""𝗣𝗿𝗶𝗰𝗲 𝗶𝘀 𝗙𝗿𝗮𝗰𝘁𝗮𝗹""\n\nIdk it will play ou...",9232,397,1941
634,2022-05-08,cjaymacy,Catching the absolute #ETH / #BTC bottom on th...,12864,3893,9525
637,2022-05-08,Eljaboom,Crypto Billionaire Brock Pierce Says Bitcoin (...,597947,1013,114075
666,2022-09-08,XForceGlobal,#DOMINANCE (BTC.D)\n\nBreaking 👀 Will this tim...,11833,148,9666


In [65]:
new_df = df.groupby('Date')

In [68]:
new_df = pd.DataFrame(new_df)
new_df

,0,1
0,2017-09-13 00:01:11+00:00,Date User ...
1,2017-09-13 00:15:01+00:00,Date User \ ...
2,2017-09-13 01:05:26+00:00,Date User \ 64 201...
3,2017-09-13 01:15:02+00:00,Date User \ ...
4,2017-09-13 01:17:06+00:00,Date User \...
...,...,...
95,2017-09-14 22:31:50+00:00,Date User ...
96,2017-09-14 23:15:01+00:00,Date User \ ...
97,2017-09-14 23:23:25+00:00,Date User \ 6...
98,2017-09-14 23:29:03+00:00,Date User ...
